#### Imports

La idea de esta variante, es predecir solamente los productos que tengan mas de 12 meses de datos (usando los ultimos 12 solamente), y aquellos con menos datos, predecir la media

In [731]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from skopt import BayesSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.metrics import MeanSquaredError
from keras.regularizers import l2
from keras.layers import BatchNormalization
from skopt.space import Categorical
from sklearn.metrics import mean_squared_error

In [732]:
final_dataset = pd.read_csv('../../Datasets/final_dataset.csv', sep='\t')

In [733]:
final_dataset.head()

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age
0,201701,20001,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
1,201702,20001,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,1
2,201703,20001,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,1,2
3,201704,20001,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,3
4,201705,20001,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,4


#### Pre procesamiento

In [734]:
final_dataset['periodo'] = pd.to_datetime(final_dataset['periodo'], format='%Y%m')

In [735]:
# Promediamos las vtas de agosto, como las de julio y setpiembre
july_september_data = final_dataset[
    final_dataset['periodo'].isin([pd.Timestamp('2019-07-01'), pd.Timestamp('2019-09-01')])
]

# Agrupar por producto y calcular el promedio
august_average = july_september_data.groupby('product_id')['y'].mean().reset_index()
august_average['periodo'] = pd.Timestamp('2019-08-01')

# Reemplazar los valores de agosto en el DataFrame original
final_dataset = final_dataset.set_index(['product_id', 'periodo'])
final_dataset.update(august_average.set_index(['product_id', 'periodo']))
final_dataset = final_dataset.reset_index()

In [736]:
# Agreog el MES del quarter, en vez de si es close_quarter
final_dataset['month'] = final_dataset['periodo'].dt.month

def month_in_quarter(month):
    if month in [1, 4, 7, 10]:
        return 1  # Primer mes del trimestre
    elif month in [2, 5, 8, 11]:
        return 2  # Segundo mes del trimestre
    elif month in [3, 6, 9, 12]:
        return 3  # Tercer mes del trimestre

final_dataset['month_in_quarter'] = final_dataset['month'].apply(month_in_quarter)

# Remover la columna 'month' si no es necesaria
final_dataset = final_dataset.drop(columns=['month'])

In [737]:
columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'month_in_quarter', 'brand', 'sku_size', 'cat1', 'cat2', 'cat3','y'] #'close_quarter'
non_scalable_columns = ['cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'plan_precios_cuidados', 'month_in_quarter'] # 'close_quarter'
clusterizable_columns=['cat1', 'cat2', 'sku_size']#, 'y']
n_features = len(columns)

Valido el promedio de agosto (este es el mes en que la empresa decidio no vender por unos dias)

In [738]:
# Filtrar los datos para los productos en julio, agosto y septiembre de 2020
august_data = final_dataset[
    final_dataset['periodo'] == pd.Timestamp('2019-08-01')
]

july_data = final_dataset[
    final_dataset['periodo'] == pd.Timestamp('2019-07-01')
]

september_data = final_dataset[
    final_dataset['periodo'] == pd.Timestamp('2019-09-01')
]

# Mostrar algunos datos de agosto para revisar que el promedio se haya aplicado
print("Datos de agosto de 2019:")
display(august_data.head())

# Mostrar algunos datos de julio para comparar
print("\nDatos de julio de 2019:")
display(july_data.head())

# Mostrar algunos datos de septiembre para comparar
print("\nDatos de septiembre de 2019:")
display(september_data.head())

# Verificar el cálculo del promedio manualmente para algunos productos
product_ids = august_data['product_id'].unique()

for product_id in product_ids[:5]:  # Limitar a los primeros 5 productos para revisar
    july_sales = july_data[july_data['product_id'] == product_id]['y'].values
    september_sales = september_data[september_data['product_id'] == product_id]['y'].values
    august_sales = august_data[august_data['product_id'] == product_id]['y'].values

    print(f"\nProducto ID: {product_id}")
    print(f"Ventas de julio: {july_sales}")
    print(f"Ventas de septiembre: {september_sales}")
    print(f"Ventas de agosto (calculadas): {august_sales}")
    if len(july_sales) > 0 and len(september_sales) > 0:
        calculated_average = (july_sales[0] + september_sales[0]) / 2
        print(f"Promedio calculado: {calculated_average}")


Datos de agosto de 2019:


,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age,month_in_quarter
31,20001,2019-08-01,0,365,1263.13762,1669.499395,HC,ROPA LAVADO,Liquido,ARIEL,3000,262.82367,0,31,2
67,20002,2019-08-01,0,315,827.28604,1078.318850,HC,ROPA LAVADO,Liquido,LIMPIEX,3000,531.76312,0,31,2
103,20003,2019-08-01,0,306,655.51370,841.487150,FOODS,ADEREZOS,Mayonesa,NATURA,475,1097.55623,0,31,2
139,20004,2019-08-01,0,304,508.31433,653.943295,FOODS,ADEREZOS,Mayonesa,NATURA,240,1284.38214,0,31,2
175,20005,2019-08-01,0,264,567.01790,812.638930,FOODS,ADEREZOS,Mayonesa,NATURA,120,751.19612,0,31,2



Datos de julio de 2019:


,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age,month_in_quarter
30,20001,2019-07-01,0,432,1782.81423,1678.99318,HC,ROPA LAVADO,Liquido,ARIEL,3000,107.73060,0,30,1
66,20002,2019-07-01,0,401,1077.79994,1066.44999,HC,ROPA LAVADO,Liquido,LIMPIEX,3000,480.50413,0,30,1
102,20003,2019-07-01,0,380,746.64283,715.20314,FOODS,ADEREZOS,Mayonesa,NATURA,475,1037.85386,0,30,1
138,20004,2019-07-01,0,359,536.82933,521.71519,FOODS,ADEREZOS,Mayonesa,NATURA,240,899.12650,0,30,1
174,20005,2019-07-01,0,372,801.30821,745.74978,FOODS,ADEREZOS,Mayonesa,NATURA,120,454.10572,0,30,1



Datos de septiembre de 2019:


,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age,month_in_quarter
32,20001,2019-09-01,0,392,1734.24194,1660.00561,HC,ROPA LAVADO,Liquido,ARIEL,3000,32.67653,1,32,3
68,20002,2019-09-01,0,438,1095.09822,1090.18771,HC,ROPA LAVADO,Liquido,LIMPIEX,3000,8.78311,1,32,3
104,20003,2019-09-01,0,470,977.10777,967.77116,FOODS,ADEREZOS,Mayonesa,NATURA,475,642.31918,1,32,3
140,20004,2019-09-01,0,466,818.26309,786.17140,FOODS,ADEREZOS,Mayonesa,NATURA,240,1562.02448,1,32,3
176,20005,2019-09-01,0,454,932.68979,879.52808,FOODS,ADEREZOS,Mayonesa,NATURA,120,1057.38804,1,32,3



Producto ID: 20001
Ventas de julio: [1678.99318]
Ventas de septiembre: [1660.00561]
Ventas de agosto (calculadas): [1669.499395]
Promedio calculado: 1669.4993949999998

Producto ID: 20002
Ventas de julio: [1066.44999]
Ventas de septiembre: [1090.18771]
Ventas de agosto (calculadas): [1078.31885]
Promedio calculado: 1078.31885

Producto ID: 20003
Ventas de julio: [715.20314]
Ventas de septiembre: [967.77116]
Ventas de agosto (calculadas): [841.48715]
Promedio calculado: 841.4871499999999

Producto ID: 20004
Ventas de julio: [521.71519]
Ventas de septiembre: [786.1714]
Ventas de agosto (calculadas): [653.943295]
Promedio calculado: 653.943295

Producto ID: 20005
Ventas de julio: [745.74978]
Ventas de septiembre: [879.52808]
Ventas de agosto (calculadas): [812.63893]
Promedio calculado: 812.6389300000001


Muetro cual es el primer mes de venta de cada producto, para debug

In [739]:
first_months = final_dataset.groupby('product_id')['periodo'].min().reset_index()
first_months.columns = ['product_id', 'first_month']

# Mostrar los resultados
print("Primer mes de cada producto:")
display(first_months.sort_values(by=['first_month', 'product_id']).tail(150))

Primer mes de cada producto:


,product_id,first_month
518,20700,2018-10-01
525,20709,2018-10-01
632,20917,2018-10-01
669,20990,2018-10-01
675,21001,2018-10-01
...,...,...
657,20962,2019-09-01
663,20975,2019-09-01
672,20995,2019-09-01
711,21087,2019-09-01


In [740]:
categorical_columns = ['cat1', 'cat2', 'cat3', 'brand', 'sku_size']

# Transformar las columnas categóricas en numéricas
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    final_dataset[col] = le.fit_transform(final_dataset[col])
    label_encoders[col] = le

#### Aislo los productos con poca historia

In [741]:
timeframe = 14

In [742]:
product_period_counts = final_dataset.groupby('product_id')['periodo'].nunique()
short_history_products = product_period_counts[product_period_counts < timeframe + 2].index

short_history_data = final_dataset[final_dataset['product_id'].isin(short_history_products)]
short_history_means = short_history_data.groupby('product_id')['y'].mean()

short_history_predictions = short_history_means.values

display(short_history_predictions)

array([5.86772106e+02, 1.86161885e+02, 2.08437450e+02, 1.19046676e+02,
       8.21773200e+01, 8.67981780e+01, 1.04676420e+02, 6.17542560e+01,
       7.96552850e+01, 5.11839650e+01, 3.51794650e+01, 2.05590953e+01,
       3.18163028e+01, 3.07301208e+01, 3.15831985e+01, 2.53293200e+01,
       1.68640175e+01, 2.60777428e+01, 1.66535563e+01, 1.60700485e+01,
       2.10238950e+01, 2.23902478e+01, 1.88899928e+01, 2.04836367e+01,
       1.80580650e+01, 1.76607944e+01, 1.47832757e+01, 2.14851800e+01,
       1.62094400e+01, 1.61332533e+01, 1.56743400e+01, 1.42599600e+01,
       1.64010521e+01, 1.82479000e+01, 1.41260867e+01, 1.32572100e+01,
       1.48035265e+01, 1.34624492e+01, 1.36020865e+01, 8.61264679e+00,
       1.32006561e+01, 2.43802350e+01, 1.15968890e+01, 1.08806433e+01,
       1.24576883e+01, 1.44772629e+01, 1.24001200e+01, 1.24984025e+01,
       9.71412000e+00, 1.94351179e+01, 9.55046600e+00, 4.35133800e+00,
       1.21669550e+01, 6.90695733e+00, 1.01741892e+01, 8.37102222e+00,
      

In [743]:
short_history_products = short_history_means.to_dict()


display(short_history_products)

{20032: 586.7721059090909,
 20085: 186.16188533333334,
 20127: 208.43745,
 20174: 119.04667642857143,
 20210: 82.17732,
 20213: 86.798178,
 20236: 104.67642000000001,
 20257: 61.754256,
 20261: 79.655285,
 20286: 51.183965,
 20355: 35.179465,
 20408: 20.559095333333335,
 20414: 31.81630277777778,
 20440: 30.730120833333334,
 20442: 31.5831985,
 20458: 25.329320000000003,
 20460: 16.8640175,
 20476: 26.077742777777775,
 20477: 16.653556333333334,
 20491: 16.070048500000002,
 20510: 21.023895,
 20521: 22.390247777777777,
 20525: 18.889992777777778,
 20526: 20.48363666666667,
 20531: 18.058065,
 20537: 17.660794444444445,
 20547: 14.783275714285713,
 20548: 21.48518,
 20558: 16.20944,
 20575: 16.133253333333332,
 20577: 15.67434,
 20580: 14.259960000000001,
 20592: 16.401052142857143,
 20593: 18.2479,
 20603: 14.126086666666668,
 20615: 13.25721,
 20620: 14.8035265,
 20621: 13.462449166666666,
 20623: 13.602086499999999,
 20627: 8.612646785714286,
 20633: 13.200656111111112,
 20649: 24.38

In [744]:
final_dataset = final_dataset[~final_dataset['product_id'].isin(short_history_products)]

#### KMEANS

In [745]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=123)
final_dataset['cluster'] = kmeans.fit_predict(final_dataset[clusterizable_columns])

In [746]:
display(final_dataset)

,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age,month_in_quarter,cluster
0,20001,2017-01-01,0,479,937.72717,934.77222,1,10,47,0,63,NaN,0,0,1,1
1,20001,2017-02-01,0,432,833.72187,798.01620,1,10,47,0,63,NaN,0,1,2,1
2,20001,2017-03-01,0,509,1330.74697,1303.35771,1,10,47,0,63,NaN,1,2,3,1
3,20001,2017-04-01,0,279,1132.94430,1069.96130,1,10,47,0,63,NaN,0,3,1,1
4,20001,2017-05-01,0,701,1550.68936,1502.20132,1,10,47,0,63,NaN,0,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20996,21245,2019-08-01,0,11,0.06824,0.04611,2,3,3,31,29,4.69333,0,11,2,0
20997,21245,2019-09-01,0,11,0.01441,0.01441,2,3,3,31,29,4.57897,1,12,3,0
20998,21245,2019-10-01,0,8,0.02691,0.02691,2,3,3,31,29,4.52997,0,13,1,0
20999,21245,2019-11-01,0,11,0.02690,0.02690,2,3,3,31,29,4.51075,0,14,2,0


#### Armos los grupos

In [747]:
grouped = final_dataset.groupby(['product_id', 'cat1', 'cat2', 'cat3'])

for name, group in grouped:
    print(f"Group name: {name}")
    print(group.head())
    print("\n")
    # break 

Group name: (20001, 1, 10, 47)
   product_id    periodo  plan_precios_cuidados  cust_request_qty  \
0       20001 2017-01-01                      0               479   
1       20001 2017-02-01                      0               432   
2       20001 2017-03-01                      0               509   
3       20001 2017-04-01                      0               279   
4       20001 2017-05-01                      0               701   

   cust_request_tn           y  cat1  cat2  cat3  brand  sku_size  \
0        937.72717   934.77222     1    10    47      0        63   
1        833.72187   798.01620     1    10    47      0        63   
2       1330.74697  1303.35771     1    10    47      0        63   
3       1132.94430  1069.96130     1    10    47      0        63   
4       1550.68936  1502.20132     1    10    47      0        63   

   stock_final  close_quarter  age  month_in_quarter  cluster  
0          NaN              0    0                 1        1  
1          

#### Aplico scalers

In [748]:
scalers = {col: {} for col in columns}
scaled_data_list = []

for (product_id, _, _, _ ), group in grouped:
    scaled_group = group.copy()
    for col in columns:
        if col not in non_scalable_columns:
            scaler = StandardScaler()
            # print(col)
            scaled_group[col] = scaler.fit_transform(group[[col]])
            scalers[col][product_id] = scaler
    scaled_data_list.append(scaled_group)

# Combinar todos los datos escalados en un solo DataFrame
scaled_data = pd.concat(scaled_data_list)

In [749]:
display(scaled_data[columns])

,plan_precios_cuidados,cust_request_qty,cust_request_tn,month_in_quarter,brand,sku_size,cat1,cat2,cat3,y
0,0,0.461891,-1.704215,1,0,63,1,10,47,-1.602615
1,0,-0.046911,-2.039835,2,0,63,1,10,47,-2.064107
2,0,0.786658,-0.435961,3,0,63,1,10,47,-0.358798
3,0,-1.703222,-1.074260,1,0,63,1,10,47,-1.146410
4,0,2.865166,0.273781,2,0,63,1,10,47,0.312213
...,...,...,...,...,...,...,...,...,...,...
20996,0,-0.491478,-0.217674,2,31,29,2,3,3,-0.285716
20997,0,-0.491478,-0.395067,3,31,29,2,3,3,-0.390061
20998,0,-0.648752,-0.353874,1,31,29,2,3,3,-0.348915
20999,0,-0.491478,-0.353907,2,31,29,2,3,3,-0.348948


#### Funcion para preparar los datos y crear el modelo

El objetivo es predecir 2 dias en el futuro, por lo que la idea es re-armar el dataset. Donde el valor de X sera el conjunto de datos hasta N-2 e Y va a ser N (siendo N la cantidad de ventas para ese producto)

In [750]:
def prepare_train_data(data):
    X, y = [], []
    unique_product_ids = data['product_id'].unique()

    for product_id in unique_product_ids:
        product_data = data[data['product_id'] == product_id].copy()
        product_values = product_data[columns].values
        
        # if len(product_values) < timeframe + 2:
        #     # if validation:
        #         # predicciones[product_id] = np.mean(product_values)
        #     continue
        
        
        X.append(product_values[-(timeframe + 2):-2])
        y.append(product_values[-1, -1])
    
    return np.array(X), np.array(y)


#### Agrupo los elemento por cluster y armo los modelos

In [751]:
# Iterar sobre cada grupo de clusters y mostrar el tamaño de cada uno
clustered_data = scaled_data.groupby('cluster')

product_id_counts = {}

# Iterar sobre cada grupo de clusters
for cluster_number, cluster_data in clustered_data:
    unique_product_ids = cluster_data['product_id'].nunique()
    product_id_counts[cluster_number] = unique_product_ids

# Imprimir el resultado
for cluster_number, count in product_id_counts.items():
    print(f"Cluster {cluster_number} tiene {count} product_id únicos")


Cluster 0 tiene 147 product_id únicos
Cluster 1 tiene 159 product_id únicos
Cluster 2 tiene 221 product_id únicos
Cluster 3 tiene 100 product_id únicos


In [752]:
# from keras.optimizers import SGD
# from keras.models import Sequential
# from keras.layers import LSTM, Dropout, Dense
# from keras.regularizers import l2
# from sklearn.model_selection import KFold
# import numpy as np
# from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# models = {}
# predictions = {}

# # clustered_data = scaled_data.groupby('cluster')

# # Definir el número de particiones para KFold
# n_splits = 5  # Puedes ajustar el número de particiones según tus necesidades
# kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# # def jitter(X, sigma=0.1):
# #     """Añadir ruido aleatorio a los datos."""
# #     noise = np.random.normal(loc=0, scale=sigma, size=X.shape)
# #     return X + noise

# # def scaling(X, sigma=0.1):
# #     """Escalar los datos aleatoriamente."""
# #     scaling_factor = np.random.normal(loc=1.0, scale=sigma, size=(X.shape[0], 1, X.shape[2]))
# #     return X * scaling_factor

# # def window_slicing(X, slice_ratio=0.9):
# #     """Recortar subsecciones aleatorias de los datos."""
# #     slice_length = int(X.shape[1] * slice_ratio)
# #     start = np.random.randint(0, X.shape[1] - slice_length + 1)
# #     return X[:, start:start + slice_length, :]

# # def augment_data(X):
# #     """Aplicar técnicas de data augmentation a los datos."""
# #     X_augmented = jitter(X)
# #     X_augmented = scaling(X_augmented)
# #     X_augmented = window_slicing(X_augmented)
# #     return X_augmented

# # Iterar sobre cada grupo de clusters
# for cluster_number, cluster_data in clustered_data:
#     print(f"Entrenando modelo para el cluster {cluster_number}")

#     # Preparar los datos para el cluster actual
#     X, y = prepare_train_data(cluster_data)
    
#     ### Augmentation
#     # X_augmented = augment_data(X)

#     # X_combined = np.concatenate((X, X_augmented), axis=0)
#     # y_combined = np.concatenate((y, y), axis=0)
#     ########

#     # Inicializar lista para guardar los resultados de cada partición
#     fold_scores = []

#     # Iterar sobre las particiones generadas por KFold
#     for fold_idx, (train_index, val_index) in enumerate(kf.split(X)):
#         print(f"Entrenando fold {fold_idx + 1}/{n_splits}")

#         X_train, X_val = X[train_index], X[val_index]
#         y_train, y_val = y[train_index], y[val_index]

#         # Crear el modelo LSTM
#         model = Sequential()
#         model.add(LSTM(128, activation='tanh', kernel_regularizer=l2(0.4), return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
#         model.add(Dropout(0.2))
#         model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.4), return_sequences=True))
#         model.add(Dropout(0.2))
#         model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.4), return_sequences=True))
#         model.add(Dropout(0.2))
#         model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.4), return_sequences=True))
#         model.add(Dropout(0.2))
#         model.add(LSTM(128, activation='relu', kernel_regularizer=l2(0.4)))
#         model.add(Dropout(0.2))
#         model.add(Dense(1))

#         # Compilar el modelo
#         optimizer = SGD(learning_rate=0.01)
#         model.compile(optimizer=optimizer, loss='mse')

#         lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
#         early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
#         model.fit(X_train, y_train, epochs=180, verbose=2, batch_size=32, validation_data=(X_val, y_val), callbacks=[lr_scheduler, early_stopping])

#         # Entrenar el modelo con los datos de entrenamiento de esta partición
#         # model.fit(X_train, y_train, epochs=150, verbose=2, batch_size=16)

#         # Evaluar el modelo con los datos de validación de esta partición
#         score = model.evaluate(X_val, y_val, verbose=0)
#         fold_scores.append(score)

#         # Opcional: Guardar el modelo en una lista o diccionario si deseas conservarlos todos

#     # Opcional: Imprimir el promedio de la métrica de evaluación (ej. MSE) de todas las particiones
#     print(f"Promedio de MSE para el cluster {cluster_number}: {np.mean(fold_scores)}")

#     # Guardar el modelo en el diccionario por cluster
#     models[cluster_number] = model


In [753]:
from keras.optimizers import SGD, RMSprop, Adam
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.regularizers import l2
from sklearn.model_selection import KFold
import numpy as np
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

models = {}
predictions = {}

### Augmentation
# def jitter(X, sigma=0.1):
#     """Añadir ruido aleatorio a los datos."""
#     noise = np.random.normal(loc=0, scale=sigma, size=X.shape)
#     return X + noise

# def scaling(X, sigma=0.1):
#     """Escalar los datos aleatoriamente."""
#     scaling_factor = np.random.normal(loc=1.0, scale=sigma, size=(X.shape[0], 1, X.shape[2]))
#     return X * scaling_factor

# def window_slicing(X, slice_ratio=0.9):
#     """Recortar subsecciones aleatorias de los datos y rellenar/cortar para mantener la misma longitud."""
#     slice_length = int(X.shape[1] * slice_ratio)
#     start = np.random.randint(0, X.shape[1] - slice_length + 1)
#     sliced_X = X[:, start:start + slice_length, :]

#     # Ajustar la longitud para que coincida con la original
#     if slice_length < X.shape[1]:
#         pad_width = X.shape[1] - slice_length
#         sliced_X = np.pad(sliced_X, ((0, 0), (0, pad_width), (0, 0)), mode='constant')
#     elif slice_length > X.shape[1]:
#         sliced_X = sliced_X[:, :X.shape[1], :]

#     return sliced_X

# def augment_data(X):
#     """Aplicar técnicas de data augmentation a los datos."""
#     X_augmented = jitter(X)
#     X_augmented = scaling(X_augmented)
#     X_augmented = window_slicing(X_augmented)
#     return X_augmented
########

# Iterar sobre cada grupo de clusters
for cluster_number, cluster_data in clustered_data:
    print(f"Entrenando modelo para el cluster {cluster_number}")

    # Preparar los datos para el cluster actual
    X, y = prepare_train_data(cluster_data)
    
    ### Augmentation
    # X_augmented = augment_data(X)

    # X_combined = np.concatenate((X, X_augmented), axis=0)
    # y_combined = np.concatenate((y, y), axis=0)
    ########

    # Inicializar lista para guardar los resultados de cada partición
    fold_scores = []

    # Iterar sobre las particiones generadas por KFold

    # Crear el modelo LSTM
    model = Sequential()
    model.add(LSTM(128, activation='tanh', kernel_regularizer=l2(0.5), return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.3))
    model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.5), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.5), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.5), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(128, activation='relu', kernel_regularizer=l2(0.5)))
    model.add(Dropout(0.3))
    model.add(Dense(1))

    # Compilar el modelo
    optimizer = Adam(learning_rate=0.01)
    model.compile(optimizer=optimizer, loss='mse')

    model.fit(X, y, epochs=200, verbose=2, batch_size=32)

    # Opcional: Imprimir el promedio de la métrica de evaluación (ej. MSE) de todas las particiones
    print(f"Promedio de MSE para el cluster {cluster_number}: {np.mean(fold_scores)}")

    # Guardar el modelo en el diccionario por cluster
    models[cluster_number] = model


4/4 - 1s - 340ms/step - loss: 0.2204
Epoch 155/200
4/4 - 1s - 297ms/step - loss: 0.2186
Epoch 156/200
4/4 - 1s - 326ms/step - loss: 0.2221
Epoch 157/200
4/4 - 1s - 333ms/step - loss: 0.2193
Epoch 158/200
4/4 - 1s - 294ms/step - loss: 0.2212
Epoch 159/200
4/4 - 1s - 326ms/step - loss: 0.2232
Epoch 160/200
4/4 - 1s - 309ms/step - loss: 0.2262
Epoch 161/200
4/4 - 1s - 350ms/step - loss: 0.2164
Epoch 162/200
4/4 - 1s - 297ms/step - loss: 0.2320
Epoch 163/200
4/4 - 1s - 350ms/step - loss: 0.2276
Epoch 164/200
4/4 - 1s - 295ms/step - loss: 0.2242
Epoch 165/200
4/4 - 1s - 345ms/step - loss: 0.2312
Epoch 166/200
4/4 - 1s - 293ms/step - loss: 0.2214
Epoch 167/200
4/4 - 1s - 303ms/step - loss: 0.2273
Epoch 168/200
4/4 - 1s - 287ms/step - loss: 0.2216
Epoch 169/200
4/4 - 1s - 276ms/step - loss: 0.2253
Epoch 170/200
4/4 - 1s - 274ms/step - loss: 0.2162
Epoch 171/200
4/4 - 1s - 311ms/step - loss: 0.2171
Epoch 172/200
4/4 - 1s - 286ms/step - loss: 0.2250
Epoch 173/200
4/4 - 1s - 309ms/step - loss: 0

/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


#### Predicciones a feb

In [754]:
def prepare_kaggle_data(data):
    X = []
    unique_product_ids = data['product_id'].unique()

    for product_id in unique_product_ids:
        product_data = data[data['product_id'] == product_id].copy()
        product_values = product_data[columns].values
        
        # if len(product_values) < timeframe:
        #     scaler = scalers['y'][(product_id)]
        #     predictions[product_id] = scaler.inverse_transform([[np.mean(product_values[:,-1])]])[0][0]
        #     continue
        
        X.append((product_id, product_values[-timeframe:]))
    
    return X

In [755]:
# Diccionario para almacenar las predicciones
predictions_dict = {}

# Prepara los datos
X_data = prepare_kaggle_data(scaled_data)

# Agrega las predicciones iniciales (para productos con historia menor al timeframe)
predictions_dict.update(short_history_products)

# Iterar sobre los productos con suficiente historia y hacer predicciones
for product_id, product_values in X_data:
    # Obtener el cluster del producto
    product_cluster = scaled_data[scaled_data['product_id'] == product_id]['cluster'].values[0]
    
    # Obtener el modelo correspondiente al cluster
    model = models[product_cluster]
    
    # Preparar los datos de entrada para la predicción
    X = product_values.reshape(1, product_values.shape[0], product_values.shape[1])
    
    # Hacer la predicción
    prediction = model.predict(X)
    
    scaler = scalers['y'][(product_id)]
    # print(prediction[0][0])
    # Guardar la predicción en el diccionario
    predictions_dict[product_id] = scaler.inverse_transform([[prediction[0][0]]])[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 800ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 827ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 850ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━

In [756]:
display(predictions_dict)

{20032: 586.7721059090909,
 20085: 186.16188533333334,
 20127: 208.43745,
 20174: 119.04667642857143,
 20210: 82.17732,
 20213: 86.798178,
 20236: 104.67642000000001,
 20257: 61.754256,
 20261: 79.655285,
 20286: 51.183965,
 20355: 35.179465,
 20408: 20.559095333333335,
 20414: 31.81630277777778,
 20440: 30.730120833333334,
 20442: 31.5831985,
 20458: 25.329320000000003,
 20460: 16.8640175,
 20476: 26.077742777777775,
 20477: 16.653556333333334,
 20491: 16.070048500000002,
 20510: 21.023895,
 20521: 22.390247777777777,
 20525: 18.889992777777778,
 20526: 20.48363666666667,
 20531: 18.058065,
 20537: 17.660794444444445,
 20547: 14.783275714285713,
 20548: 21.48518,
 20558: 16.20944,
 20575: 16.133253333333332,
 20577: 15.67434,
 20580: 14.259960000000001,
 20592: 16.401052142857143,
 20593: 18.2479,
 20603: 14.126086666666668,
 20615: 13.25721,
 20620: 14.8035265,
 20621: 13.462449166666666,
 20623: 13.602086499999999,
 20627: 8.612646785714286,
 20633: 13.200656111111112,
 20649: 24.38

In [757]:
# predictions_dict_cleaned = {product_id: max(prediction, 0) for product_id, prediction in predictions_dict.items()}
# predictions_dict_transformed = {product_id: prediction if prediction >= 0 else -1 * prediction for product_id, prediction in predictions_dict.items()}
predictions_dict_rounded = {product_id: round(prediction, 5) for product_id, prediction in predictions_dict.items()}

# Crear un DataFrame con las predicciones
predictions_df = pd.DataFrame(list(predictions_dict_rounded.items()), columns=['product_id', 'prediction'])

# Guardar el DataFrame en un archivo CSV
predictions_df.to_csv('predictions.csv', index=False)

print("Archivo CSV 'predictions.csv' guardado correctamente.")

Archivo CSV 'predictions.csv' guardado correctamente.
